In [ ]:
import json

from IPython.display import JSON, display

import ipycytoscape
import ipywidgets as W
import networkx as nx
import rdflib
from ipycytoscape import CytoscapeWidget, Edge, Node
from ipyradiant import FileManager, PathLoader
from ipyradiant.rdf2nx import RDF2NX, URItoID, URItoShortID
from rdflib.namespace import RDFS

## Start by creating a simple cytoscape graph with some node data (using the RDF to LPG conversions)

### We will use the Star Wars dataset again.

In [ ]:
lw = FileManager(loader=PathLoader(path="data"))
# here we hard set what we want the file to be, but ideally a user can choose a file to work with.
lw.loader.file_picker.value = lw.loader.file_picker.options["starwars.ttl"]
rdf_graph = lw.graph
qres = lw.graph.query(
    """
    PREFIX hum: <https://swapi.co/resource/human/>
    PREFIX film: <https://swapi.co/resource/film/>
    
    CONSTRUCT {
        ?s ?p ?o .
    }
    WHERE {
        ?s ?p ?o .
        
        VALUES (?s) {
            (hum:1)  # Luke
            (hum:4)  # Vader
            (film:1) # A New Hope
        }
    }
    """
)

simple_graph = rdflib.graph.Graph().parse(data=qres.serialize(format="xml"))
uri = RDFS.label
ns = {"rdfs": str(RDFS)}
initNs = {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
    "res": "https://swapi.co/resource/",
    "voc": "https://swapi.co/vocabulary/",
    "base": "https://swapi.co/resource/",
}

nx_graph = RDF2NX.convert(simple_graph, namespaces=initNs)
directed = ipycytoscape.CytoscapeWidget()
directed.graph.add_graph_from_networkx(nx_graph, multiple_edges=True, directed=True)
for node in directed.graph.nodes:
    # deal with inability to handle colons
    node.data["_label"] = node.data.get("rdfs:label", None)
    node.data["_attrs"] = json.dumps(node.data, indent=2)
directed.set_layout(
    name="dagre", animate=False, randomize=False, maxSimulationTime=2000
)
# Workaround for style overwriting
directed.set_style(
    [
        {
            "selector": "node",
            "css": {
                "label": "data(_label)",
                "text-wrap": "wrap",
                "text-max-width": "150px",
                "text-valign": "center",
                "text-halign": "center",
                "font-size": "10",
                "font-family": '"Gill Sans", sans-serif',
                "color": "blue",
            },
        },
        {
            "selector": "edge",
            "css": {
                "label": "data(_label)",
                "text-wrap": "wrap",
                "text-max-width": "150px",
                "text-valign": "center",
                "text-halign": "center",
                "font-size": "10",
                "font-family": '"Gill Sans", sans-serif',
                "color": "green",
            },
        },
        {
            "selector": "edge.directed",
            "style": {
                "curve-style": "bezier",
                "target-arrow-shape": "triangle",
            },
        },
        {"selector": "edge.multiple_edges", "style": {"curve-style": "bezier"}},
        {
            "selector": ":active ",
            "css": {
                "label": "data(_attrs)",
                "text-wrap": "wrap",
                "text-max-width": "500px",
                "text-valign": "bottom",
                "text-halign": "right",
                "text-background-opacity": 0.9,
                "text-background-color": "white",
                "text-background-shape": "roundrectangle",
                "color": "black",
            },
        },
    ]
)

## We now have a simple cytoscape graph with the desired style and layout characteristics.

### Let's take a look at the graph.

In [ ]:
directed

## While this graph is useful in providing relationships between nodes and subjects in the graph, a user might want to view all the specific data that has been collapsed onto the nodes in the `Cytoscape` graph. To do this, we will incorporate a JSONWidget that can be synced with the graph to show the data. This requires two steps:

1. Set up the necessary interaction events with the cytoscape graph
2. Create the output widget to display the JSON data. In this example, I'll be putting
   the JSON next to the graph in an `ipywidgets.GridBox`

In [ ]:
# set up necessary interaction events with the cytoscape graph
def log_node_clicks(node):
    full_display.selected_node = node
    data = node["data"]
    data.pop("_label", None)
    data.pop("_attrs", None)
    with json_output:
        json_output.clear_output()
        display(JSON(data))


directed.on("node", "click", log_node_clicks)

# create output widget
json_output = W.Output()
full_display = W.GridBox(
    children=[directed, json_output],
    layout=W.Layout(grid_template_columns="repeat(2, 500px)", height="450px"),
)

In [ ]:
full_display

## As seen above, we can now see the `JSON` data attached to the nodes.